In [27]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import math
import statistics
import keras
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

In [28]:
df_city_attr = pd.read_csv('./city_attributes.csv', sep=';')
df_humidity = pd.read_csv('./humidity_train.csv', sep=';')
df_pressure = pd.read_csv('./pressure_train.csv', sep=';')
df_temperature = pd.read_csv('./temperature_train.csv', sep=';')
df_weather_desc = pd.read_csv('./weather_description_train.csv', sep=';')
df_wind_dir = pd.read_csv('./wind_direction_train.csv', sep=';')
df_wind_speed = pd.read_csv('./wind_speed_train.csv', sep=';')


In [45]:
df_city_attr = pd.read_csv('./city_attributes.csv', sep=';')
df_humidity = pd.read_csv('./test/humidity_test.csv', sep=';')
df_pressure = pd.read_csv('./test/pressure_test.csv', sep=';')
df_temperature = pd.read_csv('./test/temperature_test.csv', sep=';')
df_weather_desc = pd.read_csv('./test/weather_description_test.csv', sep=';')
df_wind_dir = pd.read_csv('./test/wind_direction_test.csv', sep=';')
df_wind_speed = pd.read_csv('./test/wind_speed_test.csv', sep=';')

## Missing data replaced with average

In [3]:
real_values_col_list = df_humidity.columns[1:].tolist()

for real_col in real_values_col_list:
  df_humidity.loc[pd.isna(df_humidity.loc[:, real_col]), real_col] = df_humidity[real_col].mean()
  df_pressure.loc[pd.isna(df_pressure.loc[:, real_col]), real_col] = df_pressure[real_col].mean()
  df_temperature.loc[pd.isna(df_temperature.loc[:, real_col]), real_col] = df_temperature[real_col].mean()
  df_wind_dir.loc[pd.isna(df_wind_dir.loc[:, real_col]), real_col] = df_wind_dir[real_col].mean()
  df_wind_speed.loc[pd.isna(df_wind_speed.loc[:, real_col]), real_col] = df_wind_speed[real_col].mean()


### Missing data replaced with regression

In [29]:
DAYS_IN_YEAR = 365
DEGREES = 360

def day_of_year(dt):
  return dt.timetuple().tm_yday

real_values_col_list = df_humidity.columns[1:].tolist()

for df in [df_humidity, df_pressure, df_temperature, df_wind_dir, df_wind_speed]:
# for df in [df_humidity]:

    for real_col in real_values_col_list:

        other_real_values = real_values_col_list.copy()
        other_real_values.remove(real_col)

        df_xd = df.drop(other_real_values, 1)
        rows_with_values = df_xd[df[real_col].notnull()]
        rows_with_empty_values = df_xd[df[real_col].isnull()]

        # df_train_x = pd.DataFrame(data={'day': [day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) for dt in rows_with_values.datetime],})
        df_train_x = pd.DataFrame(data={
            'daysin': [np.sin(2 * np.pi * (day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) / DAYS_IN_YEAR)) for dt in rows_with_values.datetime],
            'daycos': [np.cos(2 * np.pi * (day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) / DAYS_IN_YEAR)) for dt in rows_with_values.datetime],
        })
        df_train_y = rows_with_values[real_col]
        # df_test_x = pd.DataFrame(data={'day': [day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) for dt in rows_with_empty_values.datetime],})
        df_test_x = pd.DataFrame(data={
            'daysin': [np.sin(2 * np.pi * (day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) / DAYS_IN_YEAR)) for dt in rows_with_empty_values.datetime],
            'daycos': [np.cos(2 * np.pi * (day_of_year(datetime.strptime(dt, "%d.%m.%Y %H:%M")) / DAYS_IN_YEAR)) for dt in rows_with_empty_values.datetime],
        })
        if(len(df_test_x) == 0):
            continue
        reg = LinearRegression()
        reg.fit(df_train_x, df_train_y)
        predicted = reg.predict(df_test_x)
        df.loc[df.loc[:, real_col].isnull(), real_col] = predicted

C:\Users\piotr\AppData\Local\Temp/ipykernel_3784/1563926460.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_xd = df.drop(other_real_values, 1)
C:\Users\piotr\AppData\Local\Temp/ipykernel_3784/1563926460.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_xd = df.drop(other_real_values, 1)
C:\Users\piotr\AppData\Local\Temp/ipykernel_3784/1563926460.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_xd = df.drop(other_real_values, 1)
C:\Users\piotr\AppData\Local\Temp/ipykernel_3784/1563926460.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_xd = df.drop(other_real_values, 1)
C:\Users\piotr\AppData\Local\Temp/ipykernel_3784

In [46]:
DAYS_IN_YEAR = 365
DEGREES = 360

def day_of_year(dt):
  return dt.timetuple().tm_yday


cities_numbers = []
count = 0
for city in df_city_attr.City:
  cities_numbers.append([city, count])
  count += 1

xd = 0

df_lists = [ [] for x in cities_numbers]

new_day = [ [df_city_attr.Latitude[i[1]], df_city_attr.Longitude[i[1]]] for i in cities_numbers]

for dt in df_humidity.datetime:
  # if(xd < 12):
  #   xd += 1
  #   continue

  datetime_object = datetime.strptime(dt, "%d.%m.%Y %H:%M")

  if(datetime_object.hour == 0):
    if(new_day):
      for i in range(len(cities_numbers)):
        df_lists[i].append(new_day[i])
    new_day = [ [df_city_attr.Latitude[x[1]], df_city_attr.Longitude[x[1]]] for x in cities_numbers]

  hum = df_humidity.loc[df_humidity.datetime == dt].values.tolist()[0]
  press = df_pressure.loc[df_pressure.datetime == dt].values.tolist()[0]
  temp = df_temperature.loc[df_temperature.datetime == dt].values.tolist()[0]
  wind_dir = df_wind_dir.loc[df_wind_dir.datetime == dt].values.tolist()[0]
  wind_dir[0] = 0
  wind_dir_sin = [np.sin(2 * np.pi * (wd / DEGREES)) for wd in wind_dir]
  wind_dir_cos = [np.cos(2 * np.pi * (wd / DEGREES)) for wd in wind_dir]
  wind_speed = df_wind_speed.loc[df_wind_speed.datetime == dt].values.tolist()[0]
  day_of_year_sin = [np.sin(2 * np.pi * (day_of_year(datetime_object) / DAYS_IN_YEAR)) for _ in wind_dir]
  day_of_year_cos = [np.cos(2 * np.pi * (day_of_year(datetime_object) / DAYS_IN_YEAR)) for _ in wind_dir]

  cities_numbers = []
  count = 0
  for city in df_city_attr.City:
    cities_numbers.append([city, count])
    count += 1


  for i in range(len(cities_numbers)):
    new_row = [
                    hum[1 + i],
                    press[1 + i],
                    temp[1 + i],
                    wind_dir_sin[1 + i],
                    wind_dir_cos[1 + i],
                    wind_speed[1 + i],
                    day_of_year_sin[i + 1],
                    day_of_year_cos[i + 1],
                    ]
    new_day[i].extend(new_row)



  # if(xd % 10 == 0):
  #   print(xd)
  xd += 1
  # if(xd >= 500):
  #   break

# dfs = pd.DataFrame(columns = ['Latitude', 'Longitude', 'Month', 'Humidity', 'Pressure',
#                              'Temp', 'WindDir', 'WindSpeed'],
#                   data = df_list)




In [57]:
NUMBER_OF_PROPS = 8
NUMBER_OF_CONSTANT_COLUMNS = 2 # Longtitude and Latitude

def prepare_one_day(day, measures_a_day):
  hours_a_measure = 24 / measures_a_day
  lists_a_measure = [[]]

  while len(day) < 24 * NUMBER_OF_PROPS + NUMBER_OF_CONSTANT_COLUMNS:
    day = [day[0], day[1]] + day[2:(2 + NUMBER_OF_PROPS)] + day[2:]

  current_idx = 0
  for j in range(24):
    lists_a_measure[current_idx] += day[NUMBER_OF_CONSTANT_COLUMNS + j * NUMBER_OF_PROPS : NUMBER_OF_CONSTANT_COLUMNS + (j + 1) * NUMBER_OF_PROPS]
    if j % hours_a_measure == hours_a_measure - 1:
      current_idx += 1
      if j < 23:
        lists_a_measure.append([])

  avg_hums = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 0]) for one_period_list in lists_a_measure]
  avg_presses = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 1]) for one_period_list in lists_a_measure]
  avg_temps = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 2]) for one_period_list in lists_a_measure]
  avg_winddirsins = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 3]) for one_period_list in lists_a_measure]
  avg_winddircoses = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 4]) for one_period_list in lists_a_measure]
  avg_windspeeds = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 5]) for one_period_list in lists_a_measure]
  avg_daysins = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 6]) for one_period_list in lists_a_measure]
  avg_daycoses = [ statistics.mean([ one_period_list[i] for i in range(len(one_period_list)) if i % NUMBER_OF_PROPS == 7]) for one_period_list in lists_a_measure]

  return avg_hums, avg_presses, avg_temps, avg_winddirsins, avg_winddircoses, avg_windspeeds, avg_daysins, avg_daycoses

def get_column_names(measures_a_day):
  columns = ['Latitude', 'Longitude']
  for i in range(measures_a_day):
    columns += ['Humidity1_' + str(i), 'Pressure1_' + str(i), 'Temp1_' + str(i), 'WindDirSin1_' + str(i), 'WindDirCos1_' + str(i), 'WindSpeed1_' + str(i), 'DayOfYearSin1_' + str(i), 'DayOfYearCos1_' + str(i),]
  for i in range(measures_a_day):
    columns += ['Humidity2_' + str(i), 'Pressure2_' + str(i), 'Temp2_' + str(i), 'WindDirSin2_' + str(i), 'WindDirCos2_' + str(i), 'WindSpeed2_' + str(i), 'DayOfYearSin2_' + str(i), 'DayOfYearCos2_' + str(i),]
  for i in range(measures_a_day):
    columns += ['Humidity3_' + str(i), 'Pressure3_' + str(i), 'Temp3_' + str(i), 'WindDirSin3_' + str(i), 'WindDirCos3_' + str(i), 'WindSpeed3_' + str(i), 'DayOfYearSin3_' + str(i), 'DayOfYearCos3_' + str(i),]
  columns += ['Temp5']
  columns += ['Wind5_reg']
  columns += ['Wind5_class']

  return columns


def prepare_data(measures_a_day = 1):

  data = []

  for single_city in df_lists:
    for i in range(len(single_city)):
      if i < 4:
        continue

      day = single_city[i]
      day1 = single_city[i - 4]
      day2 = single_city[i - 3]
      day3 = single_city[i - 2]

      if(len(day1) < 3):
        continue

      Longitude = day[0]
      Latitude = day[1]

      avg_hums1, avg_presses1, avg_temps1, avg_winddirsins1, avg_winddircoses1, avg_windspeeds1, avg_daysins1, avg_daycoses1 = prepare_one_day(day1, measures_a_day)
      avg_hums2, avg_presses2, avg_temps2, avg_winddirsins2, avg_winddircoses2, avg_windspeeds2, avg_daysins2, avg_daycoses2 = prepare_one_day(day2, measures_a_day)
      avg_hums3, avg_presses3, avg_temps3, avg_winddirsins3, avg_winddircoses3, avg_windspeeds3, avg_daysins3, avg_daycoses3 = prepare_one_day(day3, measures_a_day)
      avg_temp5 = statistics.mean([ day[2+i] for i in range(len(day[2:])) if i % NUMBER_OF_PROPS == 2])
      avg_wind5_reg = statistics.mean([ day[2+i] for i in range(len(day[2:])) if i % NUMBER_OF_PROPS == 5])
      avg_wind5_class = 1 if avg_wind5_reg > 6 else 0

      new_row = [Latitude, Longitude]

      for j in range(len(avg_hums1)):
        new_row.append(avg_hums1[j])
        new_row.append(avg_presses1[j])
        new_row.append(avg_temps1[j])
        new_row.append(avg_winddirsins1[j])
        new_row.append(avg_winddircoses1[j])
        new_row.append(avg_windspeeds1[j])
        new_row.append(avg_daysins1[j])
        new_row.append(avg_daycoses1[j])

      for j in range(len(avg_hums2)):
        new_row.append(avg_hums2[j])
        new_row.append(avg_presses2[j])
        new_row.append(avg_temps2[j])
        new_row.append(avg_winddirsins2[j])
        new_row.append(avg_winddircoses2[j])
        new_row.append(avg_windspeeds2[j])
        new_row.append(avg_daysins2[j])
        new_row.append(avg_daycoses2[j])

      for j in range(len(avg_hums3)):
        new_row.append(avg_hums3[j])
        new_row.append(avg_presses3[j])
        new_row.append(avg_temps3[j])
        new_row.append(avg_winddirsins3[j])
        new_row.append(avg_winddircoses3[j])
        new_row.append(avg_windspeeds3[j])
        new_row.append(avg_daysins3[j])
        new_row.append(avg_daycoses3[j])

      new_row.append(avg_temp5)
      new_row.append(avg_wind5_reg)
      new_row.append(avg_wind5_class)
      data.append(new_row)

  dfs = pd.DataFrame(columns = get_column_names(measures_a_day), data = data)
  return dfs

In [65]:
dfs = prepare_data(4)

In [66]:
dfs.to_csv('./pojutrze_6h_test.csv')
# dfs_class.to_csv('./dfs_class_all_24h_pojutrze.csv')

In [8]:
dfs.head(10)

,Latitude,Longitude,Humidity1_0,Pressure1_0,Temp1_0,WindDirSin1_0,WindDirCos1_0,WindSpeed1_0,DayOfYearSin1_0,DayOfYearCos1_0,...,DayOfYearCos2_0,Humidity3_0,Pressure3_0,Temp3_0,WindDirSin3_0,WindDirCos3_0,WindSpeed3_0,DayOfYearSin3_0,DayOfYearCos3_0,Temp5
0,-123.119339,49.24966,80.600412,1018.687156,284.289173,0.491135,-0.343925,1.269531,-0.999769,0.021516,...,0.038722,71.625000,1012.895719,285.528125,0.110187,-0.309902,0.541667,-0.998435,0.055917,283.757292
1,-123.119339,49.24966,77.375000,990.564947,286.145190,-0.126031,-0.336420,0.000000,-0.999250,0.038722,...,0.055917,54.594999,1019.260245,284.373333,-0.027374,0.588663,0.625000,-0.997325,0.073095,284.825833
2,-123.119339,49.24966,71.625000,1012.895719,285.528125,0.110187,-0.309902,0.541667,-0.998435,0.055917,...,0.073095,46.672499,1017.624771,283.757292,-0.103860,0.532786,0.416667,-0.995919,0.090252,285.595833
3,-123.119339,49.24966,54.594999,1019.260245,284.373333,-0.027374,0.588663,0.625000,-0.997325,0.073095,...,0.090252,43.166667,1016.434824,284.825833,0.037147,0.581649,0.375000,-0.994218,0.107381,285.943333
4,-123.119339,49.24966,46.672499,1017.624771,283.757292,-0.103860,0.532786,0.416667,-0.995919,0.090252,...,0.107381,58.059164,1018.736736,285.595833,-0.031338,0.613095,0.375000,-0.992222,0.124479,285.957292
5,-123.119339,49.24966,43.166667,1016.434824,284.825833,0.037147,0.581649,0.375000,-0.994218,0.107381,...,0.124479,64.577082,1016.307034,285.943333,-0.070290,0.571340,0.291667,-0.989932,0.141540,284.913958
6,-123.119339,49.24966,58.059164,1018.736736,285.595833,-0.031338,0.613095,0.375000,-0.992222,0.124479,...,0.141540,61.875000,1016.333104,285.957292,-0.098644,0.614262,0.583333,-0.987349,0.158559,283.828750
7,-123.119339,49.24966,64.577082,1016.307034,285.943333,-0.070290,0.571340,0.291667,-0.989932,0.141540,...,0.158559,76.440416,1017.325421,284.913958,0.468495,0.152354,1.625000,-0.984474,0.175531,283.493750
8,-123.119339,49.24966,61.875000,1016.333104,285.957292,-0.098644,0.614262,0.583333,-0.987349,0.158559,...,0.175531,77.154582,1016.174350,283.828750,0.157048,-0.419595,1.000000,-0.981306,0.192452,283.835833
9,-123.119339,49.24966,76.440416,1017.325421,284.913958,0.468495,0.152354,1.625000,-0.984474,0.175531,...,0.192452,89.648750,1017.557034,283.493750,0.073419,0.273573,0.250000,-0.977848,0.209315,285.072917


In [6]:
data_class = []

for single_city in df_lists:
  for i in range(len(single_city)):
    if i < 4:
      continue
    
    day = single_city[i]
    day1 = single_city[i - 4]
    day2 = single_city[i - 3]
    day3 = single_city[i - 2]

    Longitude = day[0]
    Latitude = day[1]

    avg_hum1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 0])
    avg_press1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 1])
    avg_temp1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 2])
    avg_winddir1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 3])
    avg_windspeed1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 4])
    avg_hum2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 0])
    avg_press2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 1])
    avg_temp2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 2])
    avg_winddir2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 3])
    avg_windspeed2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 4])
    avg_hum3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 0])
    avg_press3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 1])
    avg_temp3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 2])
    avg_winddir3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 3])
    avg_windspeed3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 4])
    max_windspeed5 = max([ day[2+i] for i in range(len(day[2:])) if i % 5 == 4])
    new_row = [Latitude, Longitude, avg_hum1, avg_press1, avg_temp1, avg_winddir1, avg_windspeed1, avg_hum2, avg_press2, avg_temp2, avg_winddir2, avg_windspeed2, avg_hum3, avg_press3, avg_temp3, avg_winddir3, avg_windspeed3, max_windspeed5]
    data_class.append(new_row)

dfs_class = pd.DataFrame(columns = ['Latitude', 'Longitude',
                              'Humidity1', 'Pressure1', 'Temp1', 'WindDir1', 'WindSpeed1',
                              'Humidity2', 'Pressure2', 'Temp2', 'WindDir2', 'WindSpeed2',
                              'Humidity3', 'Pressure3', 'Temp3', 'WindDir3', 'WindSpeed3',
                              'WindSpeed5', 
                              ],
                   data = data_class)

In [ ]:
dfs_class.head(10000).to_csv('./myfile.csv')

In [ ]:

dfs.head(10000).to_csv('./myfile.csv')

In [ ]:
df_shuffle = dfs.sample(frac = 1)

train_length = math.floor(df_shuffle.shape[0] * 0.8)

Xs = df_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_shuffle.iloc[:train_length , -1:]

In [ ]:
Xs.head(10)

,Latitude,Longitude,Humidity1,Pressure1,Temp1,WindDir1,WindSpeed1,Humidity2,Pressure2,Temp2,WindDir2,WindSpeed2,Humidity3,Pressure3,Temp3,WindDir3,WindSpeed3
6276,35.216331,31.769039,38.279200,1023.083333,288.563333,279.166667,2.208333,72.100067,1015.875000,287.225833,89.375000,1.375000,81.708333,1011.875000,290.776250,128.333333,1.500000
11346,35.216331,31.769039,72.605643,1018.500000,275.255153,154.166667,4.291667,76.454971,1009.666667,275.288604,142.208333,4.625000,60.704971,1014.333333,273.652625,262.916667,7.750000
20960,35.216331,31.769039,76.125000,1027.416667,287.001708,126.958333,1.375000,86.833333,1026.750000,287.627125,79.125000,1.791667,82.875000,1024.041667,288.437813,110.958333,1.458333
4656,35.216331,31.769039,29.500000,999.916667,305.134583,209.625000,1.333333,50.375000,998.666667,302.027271,189.875000,1.750000,50.500000,999.375000,302.064708,109.625000,1.083333
7839,35.216331,31.769039,62.666667,1026.000000,290.703435,192.250000,1.375000,60.250000,1025.375000,292.477173,147.375000,1.541667,65.375000,1029.750000,291.557271,111.875000,2.833333
19294,35.216331,31.769039,80.041667,1013.833333,290.918125,154.208333,1.833333,75.958333,1014.958333,292.031875,182.166667,1.750000,77.333333,1016.083333,293.140625,147.916667,1.625000
16623,35.216331,31.769039,68.375000,1013.875000,270.862292,148.375000,2.333333,66.541667,1011.833333,261.199792,227.208333,1.958333,60.625000,1020.625000,262.313125,280.041667,1.875000
12683,35.216331,31.769039,66.125000,1030.333333,293.220615,149.416667,1.916667,71.833333,1024.125000,294.196373,176.791667,1.333333,72.541667,1019.625000,297.232337,224.625000,1.875000
8401,35.216331,31.769039,57.541667,1018.875000,299.730042,170.916667,3.416667,57.083333,1020.291667,299.966042,184.291667,3.708333,53.791667,1024.250000,301.358125,207.458333,3.833333
3937,35.216331,31.769039,12.078153,1008.750000,301.394292,124.333333,1.708333,13.536486,1003.541667,302.542083,183.541667,3.541667,22.877811,1013.041667,297.949021,237.875000,7.125000


In [ ]:
x_train.head(5)

,Humidity1,Pressure1,Temp1,WindDir1,WindSpeed1,Humidity2,Pressure2,Temp2,WindDir2,WindSpeed2,Humidity3,Pressure3,Temp3,WindDir3,WindSpeed3
17556,-0.556803,0.527779,-2.273957,1.129473,-0.461265,-0.657942,1.156541,-2.123469,-0.065014,-0.832124,0.752563,-0.021608,-1.716158,-1.537811,1.048692
16466,0.379901,0.498904,0.404414,0.164513,-0.884717,0.414816,0.431287,0.571348,-0.115842,-0.832124,0.426037,0.504770,0.601293,-1.840135,-0.673151
21452,1.782746,0.043321,1.356596,0.913611,0.544434,0.901703,-0.380613,1.351706,0.930507,0.359603,1.221134,-0.149992,1.330899,1.736295,0.916243
10452,-0.038784,0.357738,-1.792459,-0.522161,0.650297,0.345260,0.944742,-2.595598,1.543945,1.339467,-0.621729,1.223724,-2.824410,1.117612,0.465914
11912,0.068478,0.008029,-0.662518,0.569983,-0.011347,-1.268780,0.390759,-1.196978,1.272864,0.862776,-1.079099,0.883504,-1.134093,-1.037251,-0.805600


In [ ]:
dim = x_test.shape[1]
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=100,
    decay_rate=0.9)
adam = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule)
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer=adam)

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)

In [ ]:
predicted = model.predict(x_test)
counter = 0
for i in range(len(predicted)):
  if abs(abs(predicted[i][0] -  y_test.iloc[i].values[0])) <= 2:
    counter+=1

print(counter)
print(counter/(len(predicted)))

3346
0.748211091234347


In [ ]:
df_class_shuffle = dfs_class.sample(frac = 1)

train_class_length = math.floor(df_class_shuffle.shape[0] * 0.8)

Xs = df_class_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_class_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_class_shuffle.iloc[:train_length , -1:]

#zamiana y na {0,1}
y_train.loc[y_train['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_train.loc[y_train['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1
y_test.loc[y_test['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_test.loc[y_test['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1

In [ ]:
print(y_train.head(20))

       WindSpeed5
21062         0.0
16862         1.0
5837          0.0
10758         1.0
21426         0.0
4721          0.0
16021         1.0
19684         1.0
6965          1.0
3291          1.0
2032          0.0
12682         1.0
8351          1.0
8255          0.0
58            1.0
5247          1.0
12111         0.0
8977          1.0
11461         0.0
3561          0.0


In [ ]:
dim = x_test.shape[1]
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)

Epoch 1/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5954 - accuracy: 0.6782
Epoch 2/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5951 - accuracy: 0.6773
Epoch 3/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5944 - accuracy: 0.6795
Epoch 4/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5944 - accuracy: 0.6789
Epoch 5/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5940 - accuracy: 0.6782
Epoch 6/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5937 - accuracy: 0.6808
Epoch 7/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5935 - accuracy: 0.6795
Epoch 8/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5929 - accuracy: 0.6790
Epoch 9/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5927 - accuracy: 0.6814
Epoch 10/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5925 - accuracy: 0.6796

KeyboardInterrupt: ignored